In [25]:
import numpy as np
import pandas as pd

In [26]:
path0 = r"..\01 Observational Dataset Acquisition and Processing" + "\\"

path_data1 = path0 + r"ArcticGRO\CSV" + "\\"
path_data2 = path0 + r"ArcticRIMS\CSV" + "\\"
path_data3 = path0 + r"GRDC\CSV" + "\\"
path_data4 = path0 + r"R-ArcticNET\CSV" + "\\"
path_data5 = path0 + r"WSC\CSV" + "\\"

path_data = r"Data" + "\\"
path_df = r"Table" + "\\"

file_station1 = path_df + "Station Info.csv"
file_station2 = path_df + "Station Info2.csv"

In [27]:
source_list = ["ArcticGRO", "ArcticRIMS", "GRDC", "WSC", "R-ArcticNET"]
discharge_list = ["Discharge_" + s for s in source_list]

basic_col_list = ["River", "Station", "Lon", "Lat", "Date"]

In [28]:
df_station1 = pd.read_csv(file_station1)
df_station1["River_Station"] = df_station1["River"] + "__" + df_station1["Station"]

df_station2 = pd.read_csv(file_station2)
df_station2["River_Station"] = df_station2["River"] + "__" + df_station2["Station"]

In [29]:
# Order: ArcticGRO > ArcticRIMS > GRDC > WSC > R-ArcticNET
def get_best_value(row):
    from collections import Counter

    values = pd.to_numeric(row[discharge_list], errors="coerce")
    valid_values = values.dropna().round(1)

    # Record the source of each dataset
    sources = [source_list[i] for i, val in enumerate(values) if pd.notna(val)]

    # Case 1: All are NaN
    # Case 2: Only one valid value
    if len(valid_values) <= 1:
        if len(valid_values) == 1:
            return valid_values.iloc[0], sources[0]
        else:
            return np.nan, None

    # Case 3: Two valid values, consider range/mean
    if len(valid_values) == 2:
        range_value = np.ptp(valid_values)
        mean_value = valid_values.mean()
        if mean_value == 0:  # Avoid division by zero
            return next(val for val in values if pd.notna(val)), sources[0]
        if (range_value / mean_value) < 0.05:
            return valid_values.mean(), ", ".join(sources)
        else:
            return next(val for val in values if pd.notna(val)), sources[0]

    # Case 4: Three valid values, first check if a mode appears more than once
    if len(valid_values) == 3:
        value_counts = Counter(valid_values)
        most_common = value_counts.most_common(1)[0]
        if most_common[1] >= 2:  # If a value appears 2 or more times
            return most_common[0], ", ".join(
                [
                    src
                    for val, src in zip(valid_values, sources)
                    if val == most_common[0]
                ]
            )
        # Otherwise, check range/mean
        range_value = np.ptp(valid_values)
        mean_value = valid_values.mean()
        if mean_value == 0:  # Avoid division by zero
            return next(val for val in values if pd.notna(val)), sources[0]
        if (range_value / mean_value) < 0.05:
            return valid_values.mean(), ", ".join(sources)
        else:
            return next(val for val in values if pd.notna(val)), sources[0]

    # Case 5: Four or five valid values, first check if a mode appears more than once
    if len(valid_values) >= 4:
        value_counts = Counter(valid_values)
        most_common = value_counts.most_common(1)[0]
        if most_common[1] >= 2:  # If a value appears 2 or more times
            return most_common[0], ", ".join(
                [
                    src
                    for val, src in zip(valid_values, sources)
                    if val == most_common[0]
                ]
            )

        # Detect outliers: values deviating from the mean by more than 2 standard deviations are considered outliers
        mean_value = valid_values.mean()
        std_dev = valid_values.std()
        filtered_values = valid_values[abs(valid_values - mean_value) <= 2 * std_dev]

        # If there are still 3 or more values after filtering, continue to check range/mean
        if len(filtered_values) >= 3:
            range_value = np.ptp(filtered_values)
            mean_value = filtered_values.mean()
            if mean_value == 0:  # Avoid division by zero
                return next(val for val in values if pd.notna(val)), sources[0]
            if (range_value / mean_value) < 0.05:
                return filtered_values.mean(), ", ".join(sources)
            else:
                return next(val for val in values if pd.notna(val)), sources[0]

In [30]:
for river_station in df_station1["River_Station"].unique()[:]:
    print(river_station)
    dfs = pd.DataFrame(columns=basic_col_list)

    df_rs = df_station1[df_station1["River_Station"] == river_station]
    for index, row in df_rs.iterrows():
        file = row["File"]
        df1 = pd.read_csv(file, parse_dates=["Date"])
        df1.rename(columns={"Discharge": "Discharge_" + row["Source"]}, inplace=True)
        df1.dropna(how="any", inplace=True)
        dfs = pd.merge(dfs, df1, on=basic_col_list, how="outer")

    for dis_col in discharge_list:
        if dis_col not in dfs.columns:
            dfs[dis_col] = np.nan

    dfs = dfs[basic_col_list + discharge_list]
    dfs.dropna(how="all", inplace=True)

    dfs["Discharge_Station"], dfs["Source_Discharge"] = zip(*dfs.apply(get_best_value, axis=1))
    dfs.set_index("Date", inplace=True)
    dfs = dfs.resample("D").first()
    dfs.reset_index(drop=False, inplace=True)
    dfs = dfs[basic_col_list + ["Discharge_Station", "Source_Discharge"]]
    for col in basic_col_list[:4]:
        dfs[col] = dfs[col].iloc[0]

    # Get the estuary discharge
    row2 = df_station2[df_station2["River_Station"] == river_station]
    assert len(row2) == 1
    row2 = row2.iloc[0]
    ratio = row2["Catchment_Estuary"] / row2["Catchment"]
    if ratio < 1:
        ratio = 1
    dfs["Discharge_Estuary"] = (dfs["Discharge_Station"] * ratio).round(2)

    dfs.to_csv(path_data + f"{river_station}.csv", index=False)

Anabar__Saskylakh
                                                File      Source   River  \
0  ..\01 Observational Dataset Acquisition and Pr...  ArcticRIMS  Anabar   
1  ..\01 Observational Dataset Acquisition and Pr...        GRDC  Anabar   

     Station     Lon    Lat      River_Station  
0  Saskylakh  114.08  71.97  Anabar__Saskylakh  
1  Saskylakh  114.08  71.97  Anabar__Saskylakh  
Anadyr__Snezhnoye
                                                File Source   River  \
2  ..\01 Observational Dataset Acquisition and Pr...   GRDC  Anadyr   

     Station     Lon    Lat      River_Station  
2  Snezhnoye  172.97  65.45  Anadyr__Snezhnoye  
Anderson__Below Carnwath River
                                                File Source     River  \
3  ..\01 Observational Dataset Acquisition and Pr...   GRDC  Anderson   

                Station       Lon      Lat                   River_Station  
3  Below Carnwath River -128.4133  68.6322  Anderson__Below Carnwath River  
Back__Above Her